In [1]:
import os
import tqdm
import sqlite3

In [9]:
! wget http://string90.embl.de/newstring_download/protein.links.detailed.v9.0.txt.gz
! gunzip -k protein.links.detailed.v9.0.txt.gz

string_db = "protein.links.detailed.v9.0.txt"

--2022-03-28 11:09:45--  http://string90.embl.de/newstring_download/protein.links.detailed.v9.0.txt.gz
Resolving string90.embl.de (string90.embl.de)... 194.94.44.168
Connecting to string90.embl.de (string90.embl.de)|194.94.44.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4083847181 (3.8G) [application/x-gzip]
Saving to: ‘protein.links.detailed.v9.0.txt.gz’

protein.links.detai 100%[===================>]   3.80G  2.87MB/s    in 24m 3s  

2022-03-28 11:33:49 (2.70 MB/s) - ‘protein.links.detailed.v9.0.txt.gz’ saved [4083847181/4083847181]



In [14]:
def read_in_chunks(file_object, chunk_size=1024):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data
        
chunk_size=1000000000

# string db parse
with open(string_db,"r") as f:
    string_list = []
    n = 0
    loop = tqdm.tqdm(read_in_chunks(f,chunk_size=chunk_size),leave=False)
    for chunk in loop:
        for entry in chunk:
            if n!= 0:
                entry = entry.strip().split()
                column1 = entry[0].split(".")
                taxo1 = column1[0]
                column2 = entry[1].split(".")
                taxo2 = column2[0]
                if taxo1 == "9606" and taxo2 == "9606":
                    protein1 = column1[1]
                    protein2 = column2[1]
                    neigh = int(entry[2])
                    fusion = int(entry[3])
                    co_occ = int(entry[4])
                    co_exp = int(entry[5])
                    sc_exp = int(entry[6])
                    sc_db = int(entry[7])
                    sc_txt = int(entry[8])
                    sc_all = int(entry[9])
                    string_list.append([taxo1,protein1,taxo2,protein2,neigh,fusion,co_occ,co_exp,sc_exp,sc_db,sc_txt,sc_all])
            n += 1
        

In [36]:
# Download STITCH and STRING databases
! wget --quiet http://stitch.embl.de/download/protein_chemical.links.transfer.v5.0/9606.protein_chemical.links.transfer.v5.0.tsv.gz
! gunzip -k 9606.protein_chemical.links.transfer.v5.0.tsv.gz
stitch_db = "9606.protein_chemical.links.transfer.v5.0.tsv"

#! wget --quiet https://stringdb-static.org/download/protein.links.full.v11.0/9606.protein.links.full.v11.0.txt.gz
#! gunzip -k 9606.protein.links.full.v11.0.txt.gz


#string_db = "9606.protein.links.full.v11.0.txt"

In [40]:
# stitch db parse
with open(stitch_db,"r") as f:
    stitch_list = []
    f_read = f.readlines()[1:]
    loop = tqdm.tqdm(f_read, total=len(f_read),leave=False)
    for entry in loop:
        entry = entry.strip().split()
        chiral = str(entry[0][3:4])
        cid = int(entry[0][4:])
        column2 = entry[1].split(".")
        taxo = column2[0]
        protein = column2[1]
        sc_exp = int(entry[2])
        sc_pred = int(entry[3])
        sc_db = int(entry[4])
        sc_txt = int(entry[5])
        sc_all = int(entry[6])
        stitch_list.append([chiral,cid,taxo,protein,sc_exp,sc_pred,sc_db,sc_txt,sc_all])

In [45]:
# Create sql database
stitch_columns = ["chiral","cid","taxo","protein","sc_exp","sc_pred","sc_db","sc_txt","sc_all"]
string_columns = ["tax1","pro1","tax2","pro2","neigh","fusion","co_occ","co_exp","sc_exp","sc_db","sc_txt","sc_all"]

conn = sqlite3.connect('qm_db.sqlite') 
c = conn.cursor() 

c.execute('''DROP  table IF EXISTS  stitch;''')
c.execute('''DROP  table IF EXISTS string;''')

c.execute('''CREATE TABLE stitch_protchem_man
             ([chiral] CHARACTER(1) NOT NULL, 
             [cid] integer NOT NULL, 
             [taxo] integer NOT NULL,
             [protein] VARCHAR(50) NOT NULL,
             [sc_exp] INT NOT NULL, 
             [sc_pred] INT NOT NULL,
             [sc_db] INT NOT NULL,
             [sc_txt] INT NOT NULL,
             [sc_all] INT NOT NULL
             )''')


c.execute('''CREATE TABLE string_protlink_man
             ([tax1] integer NOT NULL, 
             [pro1] VARCHAR(50) NOT NULL, 
             [tax2] integer NOT NULL,
             [pro2] VARCHAR(50) NOT NULL,
             [neigh] INT NOT NULL, 
             [fusion] INT NOT NULL,
             [co_occ] INT NOT NULL,
             [co_exp] INT NOT NULL,
             [sc_exp] INT NOT NULL,
             [sc_db] INT NOT NULL,
             [sc_txt] INT NOT NULL,
             [sc_all] INT NOT NULL
             )''')

In [46]:
# Data entry to the database
def data_entry(input_list,table):
    loop = tqdm.tqdm(input_list, total=len(input_list),leave=False)
    data_tuple_list = []
    for i,item in enumerate(loop):
        data_tuple_list.append(item)
        if i != 0 and i % 1000000 == 0:
            c.executemany("INSERT INTO " + table + " VALUES(" + str('?,' * len(item))[:-1] + ")", (data_tuple_list))
            data_tuple_list = []
            conn.commit()
    c.executemany("INSERT INTO " + table + " VALUES(" + str('?,' * len(item))[:-1] + ")", (data_tuple_list))
    conn.commit()
data_entry(stitch_list,"stitch_protchem_man")
data_entry(string_list,"string_protlink_man")

In [47]:
# Indexing the tables
c.execute("CREATE INDEX index_my_table ON stitch_protchem_man(cid,protein);")
c.execute("CREATE INDEX index_my_table2 ON string_protlink_man(pro1,pro2);")

conn.close()

In [48]:
output_folder = "../data/databases"
os.system("mkdir ../data")
os.system("mkdir " + output_folder)
os.system("mv " + stitch_db + " " + output_folder)
os.system("mv " + string_db + " " + output_folder)
os.system("mv " + stitch_db + ".gz " + output_folder)
os.system("mv " + string_db + ".gz " + output_folder)
os.system("mv qm_db.sqlite ../data")

0